In [2]:
!pip install gensim

  Using cached gensim-4.3.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (8.2 kB)
  Using cached scipy-1.13.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp310-cp310-macosx_11_0_arm64.whl (24.0 MB)
Using cached scipy-1.13.1-cp310-cp310-macosx_12_0_arm64.whl (30.3 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [scipy]
      Successfully uninstalled scipy-1.15.3━━━━━━━━━━━━━━━━━━━ 1/3 [scipy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [gensim]2m2/3 [gensim]


In [52]:
from gensim.models.keyedvectors import KeyedVectors

#以金融sgns.financial.bigram.bz2为例
model = KeyedVectors.load_word2vec_format('/Users/jing/Documents/model/sgns.weibo.bigram-char.bz2', 
                                          binary=False, 
                                          unicode_errors='ignore')


model

In [51]:
import numpy as np

def get_sentence_embedding(sentence, model):
    tokens = list(sentence)  # 如果模型是按字训练的
    vectors = [model[token] for token in tokens if token in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# 示例
sentence = "我发现了，ChatGPT出现之后，无论是就业还是用工，都变难了许多"
embedding = get_sentence_embedding(sentence, model)

In [4]:
import pandas as pd
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import jieba

# 读取数据
df = pd.read_csv('tollm_clean.csv')
df["原微博内容"] = df["原微博内容"].fillna("").astype(str)
# 构建语料库
df["分词内容"] = df["原微博内容"].apply(lambda x: " ".join(jieba.lcut(x)))
corpus = df["分词内容"].tolist()
vectorizer = TfidfVectorizer(analyzer='word')
vectorizer.fit(corpus)


TfidfVectorizer()

In [5]:
def get_sentence_embedding_tfidf(sentence, model, vectorizer):
    tokens = sentence.split()  # 这里的 sentence 是空格分词后的句子
    tfidf_vector = vectorizer.transform([sentence])
    feature_names = vectorizer.get_feature_names_out()
    word2tfidf = {feature_names[i]: tfidf_vector[0, i] for i in range(len(feature_names))}

    embeddings = []
    weights = []

    for token in tokens:
        if token in model and token in word2tfidf:
            embeddings.append(model[token] * word2tfidf[token])
            weights.append(word2tfidf[token])

    if embeddings:
        return np.sum(embeddings, axis=0) / (np.sum(weights) + 1e-9)
    else:
        return np.zeros(model.vector_size)

In [6]:
tqdm.pandas(desc="Computing TF-IDF weighted embeddings")
df["embedding"] = df["分词内容"].progress_apply(
    lambda x: get_sentence_embedding_tfidf(x, model, vectorizer)
)

# embedding_dim = model.vector_size
# embedding_matrix = np.vstack(df["embedding"].values)


Computing TF-IDF weighted embeddings: 100%|██████████| 33983/33983 [6:05:14<00:00,  1.55it/s]  


In [9]:
df.columns

Index(['MID', 'embedding'], dtype='object')

In [10]:
df.head()

,MID,embedding
0,4.841876e+15,"[0.38143182, 0.29185972, 0.23333767, 0.1143217..."
1,4.841893e+15,"[0.44457057, 0.1318372, 0.11913364, 0.2157623,..."
2,4.841901e+15,"[0.3688378, 0.2680485, 0.28779536, 0.17973045,..."
3,4.842234e+15,"[0.38237205, 0.2435549, 0.17926031, 0.01001578..."
4,4.842255e+15,"[0.22546305, 0.36613396, 0.21222816, 0.1032630..."


In [8]:
df = df[['MID', 'embedding']]

In [11]:
df.to_csv('embedding.csv', index=False)

In [ ]:
corpus = df["分词内容"].tolist()
vectorizer = TfidfVectorizer(analyzer='word')
vectorizer.fit(corpus)

In [10]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    if norm(vec1) == 0 or norm(vec2) == 0:
        return 0.0
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

In [18]:
cosine_similarity(embedding, model.get_vector('就业'))

0.32381225

In [21]:
model.similar_by_key('就业难')

# word_list = model.index_to_key
# print(len(word_list))          # 词表大小
# print(word_list[:100])         # 打印前100个词


[('就医难', 0.7379380464553833),
 ('就业', 0.7311989665031433),
 ('再就业', 0.7226001024246216),
 ('就业率', 0.7149927616119385),
 ('招工难', 0.7038132548332214),
 ('融资难', 0.6785416603088379),
 ('就业人口', 0.6658447980880737),
 ('看病难', 0.6644426584243774),
 ('守业', 0.6611716151237488),
 ('择校', 0.6508926153182983)]

In [10]:
with open('/Users/jing/Documents/model/weibo_word_list.txt', 'w', encoding='utf-8') as f:
    for word in word_list:
        f.write(word + '\n')

print(f"词表已保存，共包含 {len(word_list)} 个词。")

词表已保存，共包含 195197 个词。


# 正式使用

In [53]:
import numpy as np
words = ['就业难', '招工难', '用工荒']

# 过滤模型中不存在的词
valid_words = [word for word in words if word in model]

if not valid_words:
    raise ValueError("输入的词都不在词向量模型中。")

# 获取词向量并计算平均向量
vectors = [model[word] for word in valid_words]
center_vector = np.mean(vectors, axis=0)

In [61]:
type(center_vector)

numpy.ndarray

In [62]:
type(df2['embedding'][0])

str

In [65]:
import re
# 将错误格式字符串转为 numpy 向量的函数
def parse_embedding_string(s):
    # 去除方括号，分割空格，然后转成 float
    s_clean = re.sub(r'[\[\]]', '', s)  # 去掉中括号
    num_list = [float(x) for x in s_clean.strip().split()]
    return np.array(num_list)

# 计算余弦相似度
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# 应用
df2["embedding_array"] = df2["embedding"].apply(parse_embedding_string)
df2["y_mean"] = df2["embedding_array"].apply(lambda x: cosine_sim(x, center_vector))

/var/folders/9x/j_7cm_y109b5sg5tpf7h7gxm0000gn/T/ipykernel_74863/4078878730.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [19]:
import pandas as pd
df = pd.read_csv('tollm_clean.csv')

In [20]:
df.shape

(33983, 4)

In [24]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm  # ✅ 引入进度条库

# 定义余弦相似度
def cosine_similarity(vec1, vec2):
    if norm(vec1) == 0 or norm(vec2) == 0:
        return 0.0
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# 获取句子嵌入
def get_sentence_embedding(sentence, model):
    tokens = list(sentence)  # 如果是按词训练，用 list(jieba.cut(sentence))
    vectors = [model[token] for token in tokens if token in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# 初始化新列
df['y1'] = 0.0
df['y2'] = 0.0
df['y3'] = 0.0

# 获取关键词向量（提前）
vec1 = model.get_vector('就业难') if '就业难' in model else np.zeros(model.vector_size)
vec2 = model.get_vector('招工难') if '招工难' in model else np.zeros(model.vector_size)
vec3 = model.get_vector('用工荒') if '用工荒' in model else np.zeros(model.vector_size)

df = pd.read_csv('tollm_clean.csv')
df['原微博内容'] = df['原微博内容'].fillna('')

# 遍历数据，计算相似度（加上 tqdm 进度条）
for i, row in tqdm(df.iterrows(), total=len(df), desc="计算相似度"):
    embedding = get_sentence_embedding(row['原微博内容'], model)
    df.loc[i, 'y1'] = cosine_similarity(embedding, vec1)
    df.loc[i, 'y2'] = cosine_similarity(embedding, vec2)
    df.loc[i, 'y3'] = cosine_similarity(embedding, vec3)


计算相似度: 100%|██████████| 33983/33983 [00:17<00:00, 1950.48it/s]


In [25]:
df.head()

,MID,原微博内容,num_vn,vn_list,y1,y2,y3
0,4.841876e+15,推友@GuyP发现，#OpenAI#的最新模型ChatGPT，好像只能生成最基本的提示。Ch...,4,"['是Instruct', '进行交互', '承认错误', '尝试纠错']",0.384552,0.406587,0.400894
1,4.841893e+15,【今日新闻】#马斯克与库克误会解除#当地时间周三下午，马斯克在推特上发布了一段苹果总部的视频...,7,"['招污点', '获星1.', '跑分意外', '推出聊天版', '回答问题', '承认错误...",0.400813,0.459998,0.450493
2,4.841901e+15,OpenAI新模型来了！全新对话模型ChatGPT，可以回答用户问题，还能挑出问题的错误之处...,3,"['回答用户', '挑出问题', '承认错误']",0.393087,0.422460,0.411385
3,4.842234e+15,【让AI生成AI绘画提示词，#OpenAI最新成果ChatGPT被网友玩坏#了！还会写代码修...,15,"['写代码', '生成代码', '修复bug', '有脑洞', '设计游戏', '是AI',...",0.379966,0.415921,0.421462
4,4.842255e+15,试着和ChatGPT聊了几句，它把我的问题解决了。OOpenAI新上线GPT太强了，服务器瞬...,1,['OOpenAI新'],0.364332,0.405896,0.400272


In [26]:
df.to_csv('embedding提取y.csv', index=False)

# 合并

In [37]:
df1 = pd.read_csv('new_task_nb.csv')

In [46]:
df1.head()

,user_id,MID,time,任务数量,half_year
0,6.105753e+09,4.841876e+15,1.0,3.0,1.0
1,6.105753e+09,4.841893e+15,1.0,2.0,1.0
2,5.703922e+09,4.841901e+15,1.0,3.0,1.0
3,6.105753e+09,4.842234e+15,1.0,7.0,1.0
4,3.996876e+09,4.842255e+15,1.0,0.0,1.0


In [43]:
df1 = df1.rename(columns={'user_id_x': 'user_id'})

In [44]:
(df1['MID'][0])

4841875716178180.0

In [48]:
df2 = pd.merge(df1,df2,on='MID',how='inner')

In [71]:
df2.head()

,user_id,MID,time,任务数量,half_year,y_mean
0,6.105753e+09,4.841876e+15,1.0,3.0,1.0,0.597849
1,6.105753e+09,4.841893e+15,1.0,2.0,1.0,0.610833
2,5.703922e+09,4.841901e+15,1.0,3.0,1.0,0.514323
3,6.105753e+09,4.842234e+15,1.0,7.0,1.0,0.581837
4,3.996876e+09,4.842255e+15,1.0,0.0,1.0,0.517511


In [70]:
del df2['embedding_array']

In [67]:
df2.to_csv('embedding提取y.csv', index=False)

In [35]:
import pandas as pd
df2 = pd.read_csv('embedding.csv')

In [73]:
# 按 user_id 和 time 聚合，计算指定列的均值
agg_df = df2.groupby(['user_id', 'time'])[['y_mean', '任务数量']].mean().reset_index()


In [74]:
agg_df.head()

,user_id,time,y_mean,任务数量
0,1.044173e+09,1.0,0.560024,1.100000
1,1.044173e+09,2.0,0.625234,0.000000
2,1.044173e+09,3.0,0.597437,0.500000
3,1.088413e+09,1.0,0.503239,0.878788
4,1.088413e+09,2.0,0.498999,3.333333


In [75]:
agg_df.to_csv('2025521panel.csv',index=False)

In [27]:
df3 = pd.read_csv('514panel.csv')

In [28]:
del df3['任务数量']

In [29]:
df4 = pd.merge(agg_df, df3, on=['time', 'user_id'], how='inner')

In [30]:
df4.columns

Index(['user_id', 'time', 'y_mean', '任务数量', 'y1', 'y2', 'y3', 'Social',
       'Culture', 'Lifestyle', 'Perception', 'count', 'task_num', 'prosocial',
       'topic', '转发数', '评论数', '点赞数', '性别', '关注数', '粉丝数', '微博数', '是否认证', '认证类型',
       'avg_score', 'avg_indi', 'avg_bind', 'avg_altr', 'avg_auth', 'avg_care',
       'avg_dili', 'avg_fair', 'avg_general', 'avg_libe', 'avg_loya',
       'avg_mode', 'avg_resi', 'avg_sanc', 'avg_wast', 'Drives', 'Cognition',
       'affect', 'Relative', 'tech', 'avg_happy', 'avg_good', 'avg_surprise',
       'avg_sad', 'avg_fear', 'avg_disgust', 'avg_anger'],
      dtype='object')

In [32]:
df4['count'].value_counts()

count
1      244
2      145
3      108
4       78
6       66
      ... 
81       1
186      1
164      1
368      1
112      1
Name: count, Length: 145, dtype: int64

In [31]:
df4.to_csv('521panel.csv', index=False)